In [ ]:
import numpy as np

def generate_and_count_users(num_users, num_time_periods):
    # Generate `num_users` random numbers in the range [0, num_time_periods-1],
    # representing the time periods in which users are located
    user_time_periods = np.random.randint(0, num_time_periods, num_users)

    # Use `np.bincount` to count the number of users in each time period
    time_period_users = np.bincount(user_time_periods, minlength=num_time_periods)

    return time_period_users

# Example: 1000 users, 100 time periods
num_users = 1000
num_time_periods = 100

result = generate_and_count_users(num_users, num_time_periods)
print(result)
print(result.sum())  # Verify that the total count matches `num_users`


[ 9 22  9  9 12 11 12 13  8  5 12  5 12  7 12  5  9 12 12 14 11  9  8  4
 12  9  8 11 10 16  9 16 10  8 12 13 13  6  9  8 13  8  9 13 12 10  8 10
 11  9 14 10  8 17  3  9 12 16 14  6  8 14 12  7 13  6  9 12 12 10  8  7
  9 15 11  8  2  9  7 11 11 14  8  8  4  9  8 15 10 11  6 12 10  8 12  9
  6  8  9  8]
1000


1000


In [ ]:
import torch

# Example usage
batch_size = 2
num_users = 5
num_servers = 3

# Generate user coordinates with a structured pattern:
# The 0th user is at (0,0), the 1st user at (1,1), and so on.
# Shape: (batch_size, num_users, 2)
users_coordinates = torch.arange(num_users).view(1, -1, 1).float().repeat(batch_size, 1, 2)
print(users_coordinates)

# Generate server coordinates with a similar structured pattern as users
servers_coordinates = torch.arange(num_servers).view(1, -1, 1).float().repeat(batch_size, 1, 2)
print(servers_coordinates)

# The 0th user is assigned to the 0th server, the 1st user to the 1st server, and so on.
# For the last two users: 
# - The 3rd user experiences a distance of sqrt(3^2 + 3^2) = 3 * sqrt(2)
# - The 4th user is unassigned (represented by -1)
assigned_server_indices = torch.tensor([[0, 1, 2, 0, -1], [0, 1, 2, 0, -1]])
print(assigned_server_indices)

# Add a virtual cloud server at index -1, located at (0,0), but it won't be used in computations
servers_coordinates = torch.cat([servers_coordinates, torch.zeros(batch_size, 1, 2)], dim=1)
print(servers_coordinates)

# Mask out users assigned to index -1 so that they are excluded from computations
mask = assigned_server_indices != -1
users_coordinates = users_coordinates.masked_fill(~mask.view(batch_size, -1, 1).repeat(1, 1, 2), torch.inf)
print("masked users_coordinates", users_coordinates)

# Convert negative indices (-1) to positive indices by mapping -1 to num_servers
assigned_server_indices = torch.remainder(assigned_server_indices, num_servers + 1)

# Retrieve the assigned server coordinates based on the indices
assigned_server_coordinates = torch.gather(servers_coordinates, 1, assigned_server_indices.view(batch_size, -1, 1).repeat(1, 1, 2))
print(assigned_server_coordinates)

# Compute the Euclidean distance between users and their assigned servers
print(users_coordinates.shape, assigned_server_coordinates.shape)
distances = torch.norm(users_coordinates - assigned_server_coordinates, dim=2)
print(distances.shape)
print(distances)

# Compute user experience scores, which are inversely proportional to the square of the distance.
# - If the distance is 0, user experience = 1 + 1 = 2.
# - If the distance is 1.5, user experience = 1 + 1 / ((1.5 + 1)^2) = 1 + 1/6.25 = 1.16.
user_experiences = 1 + 1 / ((distances + 1)**2)

# Set experience scores of masked users to 0
user_experiences[~mask] = 0
print(user_experiences)


tensor([[[0., 0.],
         [1., 1.],
         [2., 2.],
         [3., 3.],
         [4., 4.]],

        [[0., 0.],
         [1., 1.],
         [2., 2.],
         [3., 3.],
         [4., 4.]]])
tensor([[[0., 0.],
         [1., 1.],
         [2., 2.]],

        [[0., 0.],
         [1., 1.],
         [2., 2.]]])
tensor([[ 0,  1,  2,  0, -1],
        [ 0,  1,  2,  0, -1]])
tensor([[[0., 0.],
         [1., 1.],
         [2., 2.],
         [0., 0.]],

        [[0., 0.],
         [1., 1.],
         [2., 2.],
         [0., 0.]]])
masked users_coordinates tensor([[[0., 0.],
         [1., 1.],
         [2., 2.],
         [3., 3.],
         [inf, inf]],

        [[0., 0.],
         [1., 1.],
         [2., 2.],
         [3., 3.],
         [inf, inf]]])
tensor([[[0., 0.],
         [1., 1.],
         [2., 2.],
         [0., 0.],
         [0., 0.]],

        [[0., 0.],
         [1., 1.],
         [2., 2.],
         [0., 0.],
         [0., 0.]]])
torch.Size([2, 5, 2]) torch.Size([2, 5, 2])
torch.Size